In [1]:
import os 
import glob 
import json 


directories = paths = ["/data0/shypula/prog_diversity/all_experiments/Open_Ended_Reevaluation_EAD_Open_and_Commercial_2024-08-02_16-02-07/", 
                        "/data1/shypula/prog_diversity/all_experiments/OpenEndedCommercialV3_2024-08-09_02-28-20/"]


key_columns = ["all_semantic_count_wcoh_nonempty_woutput", "all_average_cosine_distance_programs", "all_average_cosine_distance_raw", 
               "all_average_cosine_distance_programs_zero_null", 
               "all_ead_4_bootstrap", "all_stripped_subtrees_4_bootstrap", 
               "generations", "extracted_programs", "experiment_name", "problem_id"]


def read_tsv_file(file_path):
    """
    Read a TSV file and return all key-value pairs as a dictionary.
    """
    result_dict = {}
    with open(file_path, 'r') as f:
        for line in f:
            key, value = line.strip().split('\t')
            result_dict[key] = value
    return result_dict


experiment_dirs = []
for directory in directories:
    ds = [d for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))]
    ds = [os.path.join(directory, d) for d in ds]
    experiment_dirs.extend(ds)

all_results = []

from tqdm import tqdm
    
pbar = tqdm(total=len(experiment_dirs))
for i, directory in enumerate(experiment_dirs):
    # results_tsvs = glob.glob(os.path.join(directory, "p*", "result.tsv"))
    # if i < 5:
    #     print(f"Found {len(results_tsvs)} result.tsv files in {directory}")
    
    # for tsv_path in results_tsvs:
    #     result = read_tsv_file(tsv_path)
    #     all_results.append(result)
    experiment_name = os.path.basename(directory)
    problem_dirs = glob.glob(os.path.join(directory, "p*"))
    for problem_dir in problem_dirs:
        problem_id = os.path.basename(problem_dir)
        tsv_path = os.path.join(problem_dir, "result.tsv")
        result = read_tsv_file(tsv_path)
        generation_paths = glob.glob(os.path.join(problem_dir, "generated", "gen_*.txt"))
        extracted_program_paths = glob.glob(os.path.join(problem_dir, "generated", "prog_*.txt"))
        assert len(generation_paths) == len(extracted_program_paths) == 100, f"Expected 100 generations and extracted programs, but found {len(generation_paths)} generations and {len(extracted_program_paths)} extracted programs in {problem_dir}"
        generations = []
        extracted_programs = []
        for gen_path in generation_paths:
            with open(gen_path, 'r') as f:
                generations.append(f.read().strip())
        for prog_path in extracted_program_paths:
            with open(prog_path, 'r') as f:
                extracted_programs.append(f.read().strip())
        result["experiment_name"] = experiment_name 
        result["problem_id"] = problem_id
        result["generations"] = generations
        result["extracted_programs"] = extracted_programs
        
        all_results.append(result)
    pbar.update(1)
                
        
        
    
filtered_result_dicts = [{k: v for k, v in result.items() if k in key_columns} for result in all_results]        
    
    
    


 99%|█████████▉| 98/99 [00:07<00:00, 14.02it/s]

In [2]:
print(len(filtered_result_dicts))
for k, v in filtered_result_dicts[0].items():
    if k == "generations" or k == "extracted_programs":
        print(k, f"({len(v)} generations)")
    

2037
generations (100 generations)
extracted_programs (100 generations)


In [3]:
! pip install code-bert-score

In [6]:
import code_bert_score
prediction_idx = 0
ref_idx = 1
non_empty_results = [p for p in filtered_result_dicts[0]["extracted_programs"] if p.strip() != ""]
print(f"Found {len(non_empty_results)} non-empty programs out of {len(filtered_result_dicts[0]['extracted_programs'])} total programs")

predictions = [non_empty_results[prediction_idx]]
references = [non_empty_results[ref_idx]]

print(f"----------- Prediction -----------")
print(f"Prediction: {predictions[0]}")
print(f"----------- Reference -----------")
print(f"Reference: {references[0]}")


pred_results = code_bert_score.score(cands=predictions,
                                     refs=references,
                                     no_punc=True, lang='python')

print("----------- CodeBERT Score -----------")
print(f'Prediction precision: {pred_results[0][0]:.3f}, recall: {pred_results[1][0]:.3f}, f1: {pred_results[2][0]:.3f}, f3: {pred_results[3][0]:.3f}')

Found 39 non-empty programs out of 100 total programs
----------- Prediction -----------
Prediction: ### Output Description:




'Mac'








### Example Output:



### Test Output:
Input:  'peach apple' 

Output: 'peach, apple'

Input: 'banana cherry melon'
Output: 'banana, cherry, melon'

Input: 'apple peach banana'
Output: 'apple, banana, peach'

Input: 'banana cherry melon pineapple papaya watermelon'
Output: 'apple, banana, cherry, melon, papaya, peach, pineapple, watermelon'
----------- Reference -----------
Reference: ### Output Description:
"The cost is apples and pears overall"
"The cost of a peach is 1.25 times that of an apple."

print("\n\n\n")


tv=psutil.virtual_memory()

print(tv.total)
print(tv.used)
print(tv.free)
print(tv.percenDetail)

print(sys.getsizeof((True)))

cursor_row_results = json.loads(cursor_row_results)
print(type(cursor_row_results))
----------- CodeBERT Score -----------
Prediction precision: 0.709, recall: 0.641, f1: 0.673, f3: 0.647
